## Tensorboard Demo
tensorboard --logdir logs

In [1]:
from tensorboardX import SummaryWriter

writer = SummaryWriter("logs/exp1")

### log scalar

In [2]:
import torch

for epoch in range(100):
    loss_1 = torch.randn(1)
    loss_2 = torch.randn(1)
    total_loss = loss_1 + loss_2
#     optimizer.zero_grad()
#     total_loss.backward()
#     optimizer.step()

    writer.add_scalar('data/loss_1', loss_1.item(), epoch)
    writer.add_scalar('data/loss_2', loss_2.item(), epoch)

### log scalers

In [3]:
for epoch in range(100):
    loss_1 = torch.randn(1)
    loss_2 = torch.randn(1)
    total_loss = loss_1 + loss_2

    writer.add_scalars('data/train_loss', {'Total Loss': total_loss.item(),
                                           'Loss 1': loss_1.item(),
                                           'loss 2': loss_2.item()},
                                           epoch)

### log images

In [4]:
from torchvision import utils as vutils

for epoch in range(100):
    x = torch.rand(1, 3, 224, 224)
    if epoch % 10 == 0:
        grid = vutils.make_grid(x, normalize=True, scale_each=True)
        writer.add_image('image', x[0], epoch)

In [5]:
for epoch in range(100):
    x = torch.rand(1, 3, 224, 224)
    if epoch % 10 == 0:
        grid = vutils.make_grid(x, normalize=True, scale_each=True)
        writer.add_image_with_boxes('imagebox', x[0], torch.Tensor([[10, 10, 40, 40], [40, 40, 60, 60]]), epoch)

In [6]:
import torchvision.utils as vutils

for epoch in range(100):
    x = torch.rand(32, 3, 224, 224)
    if epoch % 10 == 0:
        grid = vutils.make_grid(x, normalize=True, scale_each=True)
        writer.add_image('image_grid', grid, epoch)
        writer.add_image_with_boxes('imagebox', x[0], torch.Tensor([[10, 10, 40, 40], [40, 40, 60, 60]]), epoch)

### log matplotlib figures

In [9]:
import matplotlib.pyplot as plt
import numpy as np

for epoch in range(10):
    fig = plt.figure()
    
    plt.plot(np.random.rand(10))
    plt.ylabel('some numbers')


    writer.add_figure('matplotlib', fig, epoch)

### model graph

In [7]:
import torch
import torch.nn as nn


class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
    
        self.conv = nn.Sequential(
            # Conv 1-1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 1-2 
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 1
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv 2-1
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 2-2 
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 2
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv 3-1
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 3-2
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 3-3
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 3
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv 4-1
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 4-2
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 4-3
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 4
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv 5-1
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 5-2
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Conv 5-3
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            # Pooling 5
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.dense = nn.Sequential(
            # Dense 6
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),

            # Dense 7
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),

            # Dense 8
            nn.Linear(4096, 1000))
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.dense(x)
        return x

model = VGG16()
dummy_input = torch.randn(1, 3, 224, 224)
writer.add_graph(model, (dummy_input, ))

In [11]:
import torch.nn.functional as F

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.bn = nn.BatchNorm2d(20)

    def forward(self, x):
        x = F.max_pool2d(self.conv1(x), 2)
        x = F.relu(x) + F.relu(-x)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = self.bn(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = Net1()

    def forward_once(self, x):
        output = self.cnn1(x)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

model = SiameseNetwork()
dummy_input = torch.rand(13, 1, 28, 28)
writer.add_graph(model, (dummy_input, dummy_input))

### histogram

In [8]:
from torchvision.models import resnet18

model = resnet18()

for name, param in model.named_parameters():
    if 'bn' not in name:
        writer.add_histogram(name, param, 0)

In [9]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.xavier_normal_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, 0)

initialize_weights(model)
for name, param in model.named_parameters():
    if 'bn' not in name:
        writer.add_histogram(name, param, 1)

In [10]:
model = resnet18(pretrained=True)

for name, param in model.named_parameters():
    if 'bn' not in name:
        writer.add_histogram(name, param, 2)

### more types

In [ ]:
# https://tensorboardx.readthedocs.io/en/latest/tensorboard.html
# writer.add_audio
# writer.add_text
# writer.add_pr_curve
# writer.add_pr_curve_raw
# writter.add_embedding
# writer.add_video